### Read txt files

In [50]:
images = "images.txt"
train_test_split = "train_test_split.txt"
classes = "classes.txt"
image_class_labels = "image_class_labels.txt"

def load_txt():
    img_dic = {}
    train_dic = {}
    class_dic = {}
    image_class_dic = {}
    
    with open(images) as img_object:
        for line in img_object:
            vals = line.replace('\n', '').split(' ')
            img_dic[vals[0]] = vals[1]
            
    with open(train_test_split) as train_object:
        for line in train_object:
            vals = line.replace('\n', '').split(' ')
            train_dic[vals[0]] = vals[1]
            
    with open(classes) as class_object:
        for line in class_object:
            vals = line.replace('\n', '').split(' ')
            key = vals[0]
            vals.remove(key)
            val = ' '.join(vals)
            class_dic[key] = (val)
            
    with open(image_class_labels) as image_class_object:
        for line in image_class_object:
            vals = line.replace('\n', '').split(' ')
            image_class_dic[vals[0]] = vals[1]
    
    image_infos = []
    for key in img_dic:
        location = img_dic[key]
        isTraining = bool(int(train_dic[key]))
        class_name = class_dic[image_class_dic[key]]
        image_infos.append([location, isTraining, class_name])
    return image_infos

def getTestInfo():
    data = [d for d in load_txt() if d[1] == True]
    return data

def getTrainInfo():
    data = [d for d in load_txt() if d[1] == False]
    return data
        
getTrainInfo()    

[['0817/0000139e21dc4d0cbfe14cae3c85c829.jpg', False, 'Oak Titmouse'],
 ['0860/0000d9fc4e024c06a0afa55cfb16b12b.jpg', False, 'Ovenbird'],
 ['0900/000193069d834334b255a447742edce3.jpg', False, 'Savannah Sparrow'],
 ['0929/000332b8997c454096472f0a8495aecf.jpg', False, 'Eastern Phoebe'],
 ['0652/000343bd521549baab9c7c97a70ac1a5.jpg',
  False,
  'Yellow-crowned Night-Heron (Immature)'],
 ['0951/0004ff8d0cc847ee94ba43352a8b9eb4.jpg', False, 'Florida Scrub-Jay'],
 ['0400/0007acfcc0e643939ab602215a82ef63.jpg', False, 'Herring Gull (Adult)'],
 ['0776/0009051ba76c4a27ac5dbf410aea7e0c.jpg',
  False,
  'Lazuli Bunting (Adult Male)'],
 ['0700/00098273c6364b0eb615b6c03b00d39a.jpg',
  False,
  'Red-tailed Hawk (Light morph immature)'],
 ['0555/000d57361eff4207a5877b8f7115d529.jpg', False, 'Red-naped Sapsucker'],
 ['0380/000ef8cc723046a08415e13eb13bbc28.jpg',
  False,
  "Allen's Hummingbird (Adult Male)"],
 ['0945/001037a2b6d14517b305c14a8bdf7963.jpg', False, "Hutton's Vireo"],
 ['0797/00114d1f804940